#   Analyse de series temporelles et Modelisation  avec  "Air Passengers "

objectif de cette analyse et de cette modélisation est de revoir la théorie des séries temporelles et d'expérimenter avec les packages R.
Nous suivrons une procédure de modélisation ARIMA de l'ensemble de données AirPassengers comme suit :
1. Effectuer une analyse exploratoire des données
2. Décomposition des données
3. Tester la stationnarité
4. Ajuster un modèle à l'aide d'un algorithme automatisé
5. Calculer les prévisions

# CHARGEMENT DES PACKAGES NECESSAIRES

In [1]:
install.packages("ggfortify")
library(ggfortify)

NameError: name 'install' is not defined

In [ ]:
install.packages("tseries")

In [ ]:
library(tseries)

In [ ]:
install.packages("forecast")

In [ ]:
library(forecast)

In [ ]:
install.packages("ggplot2")

In [ ]:
library(ggplot2)

# IMPORTATION DU JEU DE DONNEES PASSENGERS

In [ ]:
data(AirPassengers)
AP <- AirPassengers

In [ ]:
AP

In [ ]:
# Determination de la classe du jeu de données 
class(AP)

L'ensemble de données AirPassenger dans R fournit les totaux mensuels des passagers d'une compagnie aérienne américaine, de 1949 à 1960. Cet ensemble de données est déjà d'une classe de série chronologique, donc aucune autre manipulation de classe ou de date n'est requise.

# PARTIE 1: ANALYSE EXPLORATOIRE DES DONNÉES

In [ ]:
#verification de données manquantes

In [ ]:
sum(is.na(AP))

In [ ]:
# Vérification de la fréquence de la série temporelle

frequency(AP)

In [ ]:
# Verification du cycle de la serie temporelle
cycle(AP)

In [ ]:
# Consultation du résumé 
summary(AP)
#0749509930

In [ ]:
# Tracer les données brutes 
plot(AP,xlab="Date", ylab = "Passenger numbers (1000's)",main="Air Passenger numbers from 1949 to 1961")

In [ ]:
autoplot(AP) + labs(x ="Date", y = "Passenger numbers (1000's)", title="Air Passengers from 1949 to 1961")

Comme alternative à la fonction plot , nous pouvons donc également utiliser le package d'extension ggfortify R du package ggplot2, pour tracer directement à partir d'une série temporelle. Les avantages sont de ne pas avoir à convertir en une trame de données comme requis avec ggplot2, mais d'avoir toujours accès à la grammaire de superposition des graphiques.

In [ ]:
# Observation des effets saisonnières

In [ ]:
boxplot(AP~cycle(AP),xlab="Date", ylab = "Passenger Numbers (1000's)" ,main ="Monthly Air Passengers Boxplot from 1949 to 1961")

À partir de ces graphiques exploratoires, nous pouvons faire quelques premières inférences :

- Le nombre de passagers augmente au fil du temps chaque année, ce qui peut indiquer une tendance linéaire à la hausse, peut-être en raison de la demande croissante de voyages aériens et de la commercialisation des compagnies aériennes au cours de cette période.
- Dans la boîte à moustaches, il y a plus de passagers voyageant au cours des mois 6 à 9 avec des moyennes et des variances plus élevées que les autres mois, indiquant une saisonnalité avec un cycle apparent de 12 mois. La raison pour cela pourrait être qu'un plus grand nombre de personnes prennent des vacances et prennent l'avion pendant les mois d'été aux États-Unis.
- Les passagers aériens semblent être des séries chronologiques multiplicatives à mesure que le nombre de passagers augmente, il en va de même pour le modèle de saisonnalité.
- Il ne semble pas y avoir de valeurs aberrantes et il n'y a pas de valeurs manquantes. Par conséquent, aucun nettoyage des données n'est requis.

# PARTIE 2 : DECOMPOSITION DE LA SERIE TEMPORELLE

Nous allons décomposer la série chronologique pour les estimations des composantes tendancielles, saisonnières et aléatoires en utilisant la méthode de la moyenne mobile.
Le modèle multiplicatif est :

   
       Y[t]=T[t]∗S[t]∗e[t], où
       
           - Y[t] est le nombre de passager au temps t
           - T[t] est la composante de tendance au temps t,
           - S[t] est la composante saisonnière au temps t,
           - e[t] est la composante d'erreur aléatoire au temps t.

Avec ce modèle, nous utiliserons la fonction de décomposition dans R. En continuant à utiliser ggfortify pour les graphiques, sur une seule ligne, tracez automatiquement ces composants décomposés pour analyser davantage les données.

In [ ]:
decomposeAP <- decompose(AP,"multiplicative")
autoplot(decomposeAP)

Dans ces graphiques décomposés, nous pouvons à nouveau voir la tendance et la saisonnalité déduites précédemment, mais nous pouvons également observer l'estimation de la composante aléatoire représentée sous le « reste ».

# Part 3: TEST DE  STATIONARITE DE LA SERIE  TEMPORELLE

Une série chronologique stationnaire a les conditions que la moyenne, la variance et la covariance ne sont pas des fonctions du temps. Afin de s'adapter aux modèles arima, la série chronologique doit être stationnaire. Nous utiliserons deux méthodes pour tester la stationnarité.

1. Tester la stationnarité de la série temporelle (ADF)

Afin de tester la stationnarité de la série temporelle, exécutons le test  Dickey-Fuller en utilisant la fonction adf.test du package tseries R.
Commençons  par définir le test d'hypothèse :

L'hypothèse nulle H0 : la série temporelle est non stationnaire

L'hypothèse alternative H1 : la série temporelle est stationnaire

In [ ]:
adf.test(AP) 

En règle générale, lorsque la valeur p est inférieure à 5 %, nous avons des preuves solides contre l'hypothèse nulle, nous rejetons donc l'hypothèse nulle. Selon les résultats du test ci-dessus, la valeur p est de 0,01, ce qui est inférieur à  0,05. Par conséquent, nous rejetons la valeur nulle en faveur de l'hypothèse alternative selon laquelle la série chronologique est stationnaire.

2. Test de la stationnarité de la série temporelle (Autocorrélation)

Une autre façon de tester la stationnarité consiste à utiliser l'autocorrélation. Nous utiliserons la fonction d'autocorrélation (acf) dans le package R de statistiques de base. Cette fonction trace la corrélation entre une série et ses décalages c'est-à-dire les observations précédentes avec un intervalle de confiance à 95% en bleu. Si l'autocorrélation traverse la ligne bleue en pointillés, cela signifie que le décalage spécifique est significativement corrélé avec la série actuelle.

In [ ]:
autoplot(acf(AP,plot=FALSE))+ labs(title="Correlogram of Air Passengers from 1949 to 1961") 

Le maximum avec un décalage de 1 ou 12 mois, indique une relation positive avec le cycle de 12 mois.

Puisque nous avons déjà créé l'objet liste decomposeAP avec un composant aléatoire, nous pouvons tracer l'acf de decomposeAP$random.

In [ ]:
#Examiner les séries temporelles aléatoires pour toute valeur manquante
decomposeAP$random 

In [ ]:
#Autoplot de la série temporelle aléatoire à partir de 7:138 pour exclure les valeurs manquantes "NA"
autoplot(acf(decomposeAP$random[7:138],plot=FALSE))+ labs(title="Correlogram of Air Passengers Random Component from 1949 to 1961") 

On voit que l'acf des résidus est centré autour de 0.

# PARTIE 4 : MODELISATION DE LA SERIE TEMPORELLE 

1. Modèle linéaire

Puisqu'il y a une tendance à la hausse, nous examinerons d'abord un modèle linéaire à des fins de comparaison. Nous traçons l'ensemble de données brutes AirPassengers avec un modèle linéaire bleu.

In [ ]:
autoplot(AP) + geom_smooth(method="lm")+ labs(x ="Date", y = "Passenger numbers (1000's)", title="Air Passengers from 1949 to 1961")

Ce n'est peut-être pas le meilleur modèle pour s'adapter car il ne capture pas la saisonnalité et les effets multiplicatifs au fil du temps.

2. Modèle ARIMA

Utilisez la fonction auto.arima du package R de prévision pour ajuster le meilleur modèle et les meilleurs coefficients, compte tenu des paramètres par défaut, y compris la saisonnalité comme TRUE. Notez que nous avons utilisé la procédure de modélisation ARIMA comme référencé

In [ ]:
arimaAP <- auto.arima(AP)
arimaAP

Les paramètres du modèle ARIMA(2,1,1)(0,1,0)[12] sont la différenciation de décalage 1 (d), un terme autorégressif de second décalage (p) et un modèle de moyenne mobile d'ordre 1 (q). Ensuite, le modèle saisonnier a un terme autorégressif de premier décalage (D) à la période du modèle 12 unités, dans ce cas des mois.

# Le modele arima est :
 
                  Ypredit= 0.5960Y_(t-2)+0.2143Y_(t-12)-0.9819e_(t-1)+E
                  
              Où E est une erreur.
              
  La fonction ggtsdiag du package ggfortify R effectue des diagnostics de modèle des résidus et de l'acf. comprendra un   d'autocovariance.                
              

In [ ]:
install.packages("ggtsdiag")

# Part 5: CALCULONS LES PRÉVISIONS

Enfin, nous pouvons tracer une prévision de la série chronologique à l'aide de la fonction de prévision, toujours à partir du package de prévision R, avec un intervalle de confiance de 95 % où h correspond aux périodes de l'horizon de prévision en mois

In [ ]:
forecastAP <- forecast(arimaAP, level = c(95), h = 36)
autoplot(forecastAP)